# clover-on-clover connected proton charge radius

In [1]:
import numpy as np
import pandas as pd
import gvar as gv
import matplotlib.pyplot as plt
%matplotlib notebook
from tqdm import tqdm_notebook as tqdm

In [58]:
# Some useful functions
def plot_2pt_z(data,title='2pt z-meff'):
    # two point
    z = np.arange(1,32)
    trange = [2,10]
    fig = plt.figure(title,figsize=(7,4.326237))
    ax = plt.axes([0.15,0.15,0.8,0.8])
    for t in range(trange[0],trange[1]+1):
        pltdata = gv.dataset.avg_data(data[:,t,:])
        #pltdata = (pltdata + np.roll(pltdata[::-1],1))/2 # fold data
        meff = pltdata[z] #np.arccosh((np.roll(pltdata,1)+np.roll(pltdata,-1))[z]/(2*pltdata[z]))
        ax.errorbar(z,y=[i.mean for i in meff],yerr=[i.sdev for i in meff])
    ax.set_yscale("log")
    plt.draw()
def plot_3pt_z(data,title='gV'):
    # three point
    z = np.arange(2,10)
    trange = [0,5]
    fig = plt.figure(title,figsize=(7,4.326237))
    ax = plt.axes([0.15,0.15,0.8,0.8])
    for t in range(trange[0],trange[1]+1):
        pltdata = gv.dataset.avg_data(data[:,t,:])
        pltdata = (pltdata + np.roll(pltdata[::-1],1))/2 # fold data
    ax.errorbar(z,y=[i.mean for i in pltdata],yerr=[i.sdev for i in pltdata])
    ax.set_yscale("log")
    plt.draw()

# read csv file into dataframe
## ensemble information
+ 0.12fm lattice spacing
+ pion mass 400 MeV
+ isoclover

## configuration information
+ 1000 to 2950 skip 10 (196 configurations)
    + configuration 1470 is missing for baryon 2pt tsrc 48, idx = 47
+ two point correlation function
    + `bar_s22_SS_tsrc_[0,48].dat.[re,im]`
    + two sources at `[0,48]`
    + each line contains:
        + individual momentum `[0,1,-1,2,-2]`
        + individual time slice
        + array of z-dependence
        + *e.g.* `bar_s22_S0src_S0_px0_py0_pz0_t0_re`
+ three point correlation function
    + `barff_s22_SS_g8_T[10,14]_tsrc_[0,48].dat.[re,im]`
    + `g8` corresponds to the `V4` current insertion
    + `T[10,14]` is snk location
    + two sources at `[0,48]`
    + each line contains:
        + individual momentum transfer (snk at zero momentum) `[0,1,-1,2,-2]`
        + individual time slice
        + array of z-dependence at current insertion
        + *e.g.* `barff_s2_S0src_S0snk_T10_px0_py0_pz0_g8_t0_re`

## strategy
+ read zero momentum spatial correlator
+ perform source average
+ construct two and three point (with momentum transfer)
+ construct moment correlators

## notes
+ read SS src snk

In [10]:
# data parameters
dataset2 = 'bar_s22_SS_tsrc_SRC.dat.re'
dataset3 = 'barff_s22_SS_g8_TSNK_tsrc_SRC.dat.re'
zdata = dict()
# read two point
dataset = list()
for s in [0,48]: # automatically read both sources
    rd = dataset2.replace('SRC',str(s))
    df = pd.read_csv('./data/%s' %rd,delimiter=' ',header=None)
    tag = df[0][0]
    key =tag.replace('t0_','')
    # read zero momentum spatial correlator
    corr = np.array([np.array(df.loc[df[0] == tag.replace('t0','t%s' %t)].as_matrix())[:,1:-1] for t in range(96)])
    corr = np.swapaxes(corr,0,1) # swap axis to get [cfg,t,z]
    # delete missing config 1470 from other datasets
    if len(corr) == 196:
        corr = np.delete(corr,47,axis=0)
    dataset.append(corr)
    # average sources
zdata[key] = (dataset[0]+dataset[1])/2
print('2pt:',key,np.shape(zdata[key]))
# read three point
for snk in [10,14]:
    dataset=list()
    for s in [0,48]:
        rd = dataset3.replace('SRC',str(s)).replace('SNK',str(snk))
        df = pd.read_csv('./data/%s' %rd,delimiter=' ',header=None)
        tag = df[0][0]
        key = tag.replace('t0_','')
        # read zero momentum spatial correlator
        corr = np.array([np.array(df.loc[df[0] == tag.replace('t0','t%s' %t)].as_matrix())[:,1:-1] for t in range(96)])
        corr = np.swapaxes(corr,0,1) # swap axis to get [cfg,t,z]
        # delete missing config 1470 from other datasets
        if len(corr) == 196:
            corr = np.delete(corr,47,axis=0)
        dataset.append(corr)
    # average sources
    zdata[key] = (dataset[0]+dataset[1])/2
    print('3pt:',key,np.shape(zdata[key]))

2pt: bar_s22_S0src_S0_px0_py0_pz0_re (195, 96, 32)
3pt: barff_s2_S0src_S0snk_T10_px0_py0_pz0_g8_re (195, 96, 32)
3pt: barff_s2_S0src_S0snk_T14_px0_py0_pz0_g8_re (195, 96, 32)


# construct two and three point correlators
+ Lattice 2016 proceedings @ https://arxiv.org/pdf/1610.02354.pdf
+ project to definite momentum

In [21]:
mom = [0,1,2] # momentum transfer
data = dict()
# two point correlation function
for n in mom:
    k = 2.*np.pi*n/32
    pos = zdata['bar_s22_S0src_S0_px0_py0_pz0_re']*np.cos(k*np.arange(32))
    neg = zdata['bar_s22_S0src_S0_px0_py0_pz0_re']*np.cos(-k*np.arange(32))
    data['nucleon_q%s' %n] = np.sum((pos+neg)/2,axis=2) # mom avg & project to 3 momentum
# gV correlation function
for n in mom:
    k = 2.*np.pi*n/32
    pos = zdata['barff_s2_S0src_S0snk_T10_px0_py0_pz0_g8_re']*np.cos(k*np.arange(32))
    neg = zdata['barff_s2_S0src_S0snk_T10_px0_py0_pz0_g8_re']*np.cos(-k*np.arange(32))
    data['gV_T10_q%s' %n] = np.sum((pos+neg)/2,axis=2) # mom avg & project to 3 momentum
    pos = zdata['barff_s2_S0src_S0snk_T14_px0_py0_pz0_g8_re']*np.cos(k*np.arange(32))
    neg = zdata['barff_s2_S0src_S0snk_T14_px0_py0_pz0_g8_re']*np.cos(-k*np.arange(32))
    data['gV_T14_q%s' %n] = np.sum((pos+neg)/2,axis=2) # mom avg & roject to 3 momentum
data.keys()

dict_keys(['nucleon_q0', 'nucleon_q1', 'nucleon_q2', 'gV_T10_q0', 'gV_T14_q0', 'gV_T10_q1', 'gV_T14_q1', 'gV_T10_q2', 'gV_T14_q2'])

In [18]:
rz2 = np.roll((np.arange(32)-15)**2,17)
print(np.shape(zdata['bar_s22_S0src_S0_px0_py0_pz0_re']*0.5*rz2))
print(np.abs(np.roll(np.arange(32)-15,17))**2)

(195, 96, 32)
[  0   1   4   9  16  25  36  49  64  81 100 121 144 169 196 225 256 225
 196 169 144 121 100  81  64  49  36  25  16   9   4   1]


# construct moments of correlation functions

In [63]:
# two point moment data
z = np.roll(np.arange(32)-15,17)
rz2 = z**2
# apply moment
# output is Ncf * T * Z
for n in mom:
    if n == 0:
        data['dnucleon_q0'] = 0.5*rz2*zdata['bar_s22_S0src_S0_px0_py0_pz0_re']
        data['dgV_T10_q0'] = 0.5*rz2*zdata['barff_s2_S0src_S0snk_T10_px0_py0_pz0_g8_re']
        data['dgV_T14_q0'] = 0.5*rz2*zdata['barff_s2_S0src_S0snk_T14_px0_py0_pz0_g8_re']
        #plot_2pt_z(data['dgV_T10_q0'],title='p1')
    else:
        k = 2.*np.pi*n/32
        data['dnucleon_q%s' %n] = z/(2.*k)*np.sin(k*np.arange(32))*zdata['bar_s22_S0src_S0_px0_py0_pz0_re']
        data['dgV_T10_q%s' %n] = z/(2.*k)*np.sin(k*np.arange(32))*zdata['barff_s2_S0src_S0snk_T10_px0_py0_pz0_g8_re']
        data['dgV_T14_q%s' %n] = z/(2.*k)*np.sin(k*np.arange(32))*zdata['barff_s2_S0src_S0snk_T14_px0_py0_pz0_g8_re']
        #plot_2pt_z(data['dgV_T10_q1'],title='p2')
# project 3 momentum (didn't do it before for plotting purposes)
for n in mom:
    data['dnucleon_q%s' %n] = np.sum(data['dnucleon_q%s' %n],axis=2)
    data['dgV_T10_q%s' %n] = np.sum(data['dgV_T10_q%s' %n],axis=2)
    data['dgV_T14_q%s' %n] = np.sum(data['dgV_T14_q%s' %n],axis=2)

# construct correlated dataset

In [64]:
gvdata = gv.dataset.avg_data(data)
for k in gvdata.keys():
    print(k,np.shape(gvdata[k]))

nucleon_q0 (96,)
nucleon_q1 (96,)
nucleon_q2 (96,)
gV_T10_q0 (96,)
gV_T14_q0 (96,)
gV_T10_q1 (96,)
gV_T14_q1 (96,)
gV_T10_q2 (96,)
gV_T14_q2 (96,)
dnucleon_q0 (96,)
dgV_T10_q0 (96,)
dgV_T14_q0 (96,)
dnucleon_q1 (96,)
dgV_T10_q1 (96,)
dgV_T14_q1 (96,)
dnucleon_q2 (96,)
dgV_T10_q2 (96,)
dgV_T14_q2 (96,)


# plot correlation function

In [74]:
fig = plt.figure('meff',figsize=(7,4.326237))
ax = plt.axes([0.15,0.15,0.8,0.8])
x = np.arange(2,18)
meff = np.log(gvdata['dnucleon_q1']/np.roll(gvdata['dnucleon_q1'],-1))[x]
ax.errorbar(x,y=[i.mean for i in meff],yerr=[i.sdev for i in meff])
meff = np.log(gvdata['nucleon_q1']/np.roll(gvdata['nucleon_q1'],-1))[x]
ax.errorbar(x,y=[i.mean for i in meff],yerr=[i.sdev for i in meff])
plt.draw()
fig = plt.figure('correlator',figsize=(7,4.326237))
ax = plt.axes([0.15,0.15,0.8,0.8])
x = np.arange(0,12)
meff = gvdata['gV_T10_q0'][x]
ax.errorbar(x,y=[i.mean for i in meff],yerr=[i.sdev for i in meff])
meff = gvdata['dgV_T10_q0'][x]
ax.errorbar(x,y=[i.mean for i in meff],yerr=[i.sdev for i in meff])
plt.draw()

<IPython.core.display.Javascript object>

/Users/cchang5/anaconda/envs/pyqcd3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.
/Users/cchang5/anaconda/envs/pyqcd3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log
  


<IPython.core.display.Javascript object>

In [7]:
197/0.118*0.72

1202.033898305085

In [27]:
197/0.12*0.84

1379.0